# Functions

## How functions work

A function returns a value based on the given arguments:

In [2]:
def add_one(number):
    return number + 1

add_one(2)

3

or it may return None and just print the information as print() does.

In general, functions in Python may also have side effects rather than just turning an input into an output. The print() function is a basic example of this: it returns None while having the side effect of outputting something to the console. However, to understand decorators, it is enough to think about functions as something that turns given arguments into a value.

## First-Class Objects

In [8]:
def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def greet_bob(greeter_func, name):
    return greeter_func(name)

In [10]:
greet_bob(say_hello, 'Jane')

'Hello Jane'

In [11]:
greet_bob(be_awesome, 'Jane')

'Yo Jane, together we are the awesomest!'

In both cases we pass the name of the functions without parenthesis - `say_hello` and `be_awesome`, that means only a reference to the function is passed.

## Inner Functions

It’s possible to define functions inside other functions. Such functions are called inner functions.

In [14]:
def parent():
    print("Printing from the parent() function")

    def first_child():
        print("Printing from the first_child() function")

    def second_child():
        print("Printing from the second_child() function")

    second_child()
    first_child()

parent()

Printing from the parent() function
Printing from the second_child() function
Printing from the first_child() function


## Returning Functions From Functions

In [15]:
def parent(num):
    
    def first_child():
        return "Hi, I am Emma"

    def second_child():
        return "Call me Liam"

    if num == 1:
        return first_child
    else:
        return second_child

Note that we are returning first_child without the parentheses. Recall that this means that you are returning a reference to the function first_child. In contrast first_child() with parentheses refers to the result of evaluating the function. This can be seen in the following example:

In [19]:
first = parent(1)
second = parent(2)
first, second

(<function __main__.parent.<locals>.first_child()>,
 <function __main__.parent.<locals>.second_child()>)

In [20]:
first()

'Hi, I am Emma'

## Simple Decorators

In [24]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

# The so-called decoration happens at the following line: 
say_whee = my_decorator(say_whee)

say_whee

<function __main__.my_decorator.<locals>.wrapper()>

### Put simply: decorators wrap a function, modifying its behavior.

In [28]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper

def say_whee():
    print("Whee!")

say_whee = not_during_the_night(say_whee)
say_whee()

Whee!


In [33]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper
    
@not_during_the_night
def say_whee():
    print("Whee!")

In [32]:
say_whee()

### We can import decorators from other files like normal functions. 

Create a file called decorators.py with the following content:

In [34]:
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

In [67]:
# from decorators import do_twice

@do_twice
def say_whee():
    print("Whee!")

## Decorating Functions With Arguments

In [38]:
@do_twice
def greet(name):
    print(f"Hello {name}")
    
greet('World')

TypeError: wrapper_do_twice() takes 0 positional arguments but 1 was given

The problem is that the inner function `wrapper_do_twice()` does not take any arguments, but name=`"World"` was passed to it. You could fix this by letting `wrapper_do_twice()` accept one argument, but then it would not work for the `say_whee()` function you created earlier.

The solution is to use `*args` and `**kwargs` in the inner wrapper function. Then it will accept an arbitrary number of positional and keyword arguments.

In [39]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

In [42]:
@do_twice
def greet(name):
    print(f"Hello {name}")
    
greet('World')

Hello World
Hello World


In [43]:
say_whee()

Whee!
Whee!


## Returning Values From Decorated Functions

In [48]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

hi_jane = return_greeting('Jane')
print(hi_jane)

Creating greeting
Creating greeting
None


Oops, your decorator ate the return value from the function.

Because the do_twice_wrapper() doesn’t explicitly return a value, the call return_greeting("Jane") ended up returning None.

To fix this, you need to make sure the wrapper function returns the return value of the decorated function. Change your decorators.py file:

In [49]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [52]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

hi_jane = return_greeting('Jane')
print(hi_jane)

Creating greeting
Creating greeting
Hi Jane


## Who Are You, Really?

A great convenience when working with Python, especially in the interactive shell, is its powerful introspection ability. Introspection is the ability of an object to know about its own attributes at runtime. For instance, a function knows its own name and documentation:

In [53]:
print

<function print>

In [54]:
print.__name__

'print'

In [55]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



The introspection works for functions you define yourself as well:

In [56]:
say_whee

<function __main__.do_twice.<locals>.wrapper_do_twice()>

In [57]:
say_whee.__name__

'wrapper_do_twice'

In [60]:
help(say_whee)

Help on function wrapper_do_twice in module __main__:

wrapper_do_twice()



However, after being decorated, `say_whee()` has gotten very confused about its identity. It now reports being the `wrapper_do_twice()` inner function inside the `do_twice()` decorator. Although technically true, this is not very useful information.

To fix this, decorators should use the `@functools.wraps` decorator, which will preserve information about the original function. Update decorators.py again:

In [64]:
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [68]:
say_whee

<function __main__.say_whee()>

In [69]:
say_whee.__name__

'say_whee'

In [70]:
help(say_whee)

Help on function say_whee in module __main__:

say_whee()



Much better! Now `say_whee()` is still itself after decoration.

### Technical Detail: 
The **@functools.wraps** decorator uses the function `functools.update_wrapper()` to update special attributes like `__name__` and `__doc__` that are used in the introspection.

## A Few Real World Examples

In [72]:
# This formula is a good boilerplate template for building more complex decorators.

import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator

### Timing Functions

[difference b/n time.perf_counter() and time.process_time()](https://www.webucator.com/blog/2015/08/python-clocks-explained/)

In [80]:
import functools
import time

def timer(func):
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.process_time()
        value = func(*args, **kwargs)
        end_time = time.process_time()
        run_time = end_time - start_time
        print(f"Elapsed time: {run_time:0.4f} seconds")
        return value
    return wrapper_timer

In [81]:
@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [78]:
waste_some_time(1)

Elapsed time: 0.0156 seconds


In [79]:
waste_some_time(1000)

Elapsed time: 2.6719 seconds


In [82]:
waste_some_time(1000)

Elapsed time: 2.7308 seconds
